In [ ]:
import numpy as np
import pandas as pd
import time

In [ ]:
# read in meta data
metadata = pd.read_csv("data\inst_tuning\heur040_n_300_m_13358.txt", sep=" ", nrows=1, header=None).iloc[0]
s = metadata.iloc[0]
n = metadata.iloc[1]
m = metadata.iloc[2]
l = metadata.iloc[3]

n

In [ ]:
df = pd.read_csv("data\inst_tuning\heur040_n_300_m_13358.txt", sep=" ", skiprows=1, names = ["n1", "n2", "e", "w"])
df

In [ ]:
# create nodes data frame
nodes_from = df.loc[df["e"]==1][["n1","w"]].groupby(['n1']).sum()
nodes_from
nodes_to = df.loc[df["e"]==1][["n2","w"]].groupby(['n2']).sum()
nodes_to

nodes = nodes_from.join(nodes_to, lsuffix='_from', rsuffix='_to', how = 'outer')
nodes['node_impact'] = nodes.w_from.fillna(0) + nodes.w_to.fillna(0)
nodes = nodes.drop(columns=['w_from', 'w_to'])
nodes['current_degree'] = 0
nodes['splex'] = nodes.index
nodes = nodes.reset_index().rename(columns={"index":"node_number"})
nodes

In [ ]:
edges = df.copy()
edges['w'] = edges['w'] * (1-(edges['e']*2))
edges['e'] = 0
edges.loc[edges['w'] < 0]

In [ ]:
def is_splex(nodes, plex_number, s) -> bool | pd.DataFrame:
    splex = nodes.loc[nodes["splex"] == plex_number]
    min_degree = len(splex.index) - s
    problem_nodes = splex.loc[splex["current_degree"] < min_degree]
    
    if len(problem_nodes.index) == 0:
        return True
    else:
        return problem_nodes

is_splex(nodes, 2, 1)

In [ ]:
existing_edges = df.loc[df["e"]==1].sort_values("w", ascending=False)
existing_edges

In [ ]:
nodes_in_plex = nodes.loc[(nodes["splex"]==201) | (nodes["splex"] ==30), "node_number"].values
nodes_in_plex
#for node in nodes_in_plex:
#    print(nodes.loc[nodes["node_number"]==node,"current_degree"].values[0]<3)

In [ ]:
existing_edges = df.loc[df["e"]==1].sort_values("w", ascending=False)

remaining_edges = len(existing_edges) # wozu?
start = time.time()
for index, row in existing_edges.iterrows():
    # get plex assignment of both nodes
    n1_plex = nodes.loc[nodes["node_number"]==row["n1"], "splex"].values[0]
    n2_plex = nodes.loc[nodes["node_number"]==row["n2"], "splex"].values[0]
    # all nodes that would be in the plex if we merged it
    nodes_in_plex = nodes.loc[(nodes["splex"]==n1_plex) | (nodes["splex"] ==n2_plex), "node_number"].values
    # I guess this is for the TODO
    other_edges_to_add = existing_edges.loc[(existing_edges["n1"].isin(nodes_in_plex)) & 
                                            (existing_edges["n2"].isin(nodes_in_plex))]
    
    
    number_of_nodes = len(nodes_in_plex)
    edges_missing = False

    for node in nodes_in_plex:
        # check if it would be a valid splex if we merge
        if nodes.loc[nodes["node_number"]==node,"current_degree"].values[0] < (number_of_nodes - s):
            edges_missing = True
            # TODO: Add other edges
    
    # if it would be a valid splex, actually merge it
    if not edges_missing:
        # merge them
        nodes.loc[nodes["splex"]==n2_plex, "splex"] = n1_plex
        # include the connectig edge
        edges.loc[(edges["n1"]==row["n1"]) & (edges["n2"]==row["n2"]), "e"] = 1 
        # update node info (degree and node impact)
        nodes.loc[nodes["node_number"]==row["n1"], "current_degree"] +=1
        nodes.loc[nodes["node_number"]==row["n2"], "current_degree"] +=1
        nodes.loc[nodes["node_number"]==row["n1"], "node_impact"] -=row["w"]
        nodes.loc[nodes["node_number"]==row["n2"], "node_impact"] -=row["w"]

        remaining_edges -= 1 #wozu?
print(round(time.time()-start, 2), "seconds")

In [ ]:
nodes

In [ ]:
edges.loc[edges['e'] >0]